In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from bisect import bisect_right
import functions as f
import winsound
import time
import itertools

def shift_time(timestamp_str, minutes=0, seconds=0, hours=0):
    # Only time (HH:MM:SS)
    if len(timestamp_str) == 8:
        h = int(timestamp_str[0:2])
        m = int(timestamp_str[3:5])
        s = int(timestamp_str[6:8])
        total = h * 3600 + m * 60 + s + hours * 3600 + minutes * 60 + seconds
        total %= 86400  # Wrap around 24h
        return f"{total // 3600:02}:{(total % 3600) // 60:02}:{total % 60:02}"

    # Only date (YYYY-MM-DD)
    elif len(timestamp_str) == 10:
        y, mo, d = map(int, timestamp_str.split("-"))
        t = timedelta(hours=hours, minutes=minutes, seconds=seconds)
        dt = datetime(y, mo, d) + t
        return dt.strftime("%Y-%m-%d")

    # Full datetime (YYYY-MM-DD HH:MM:SS)
    else:
        y = int(timestamp_str[0:4])
        mo = int(timestamp_str[5:7])
        d = int(timestamp_str[8:10])
        h = int(timestamp_str[11:13])
        m = int(timestamp_str[14:16])
        s = int(timestamp_str[17:19])
        total = (datetime(y, mo, d, h, m, s) +
                 timedelta(hours=hours, minutes=minutes, seconds=seconds))
        return total.strftime("%Y-%m-%d %H:%M:%S")

In [2]:
def calculate_candle_gains(df: pd.DataFrame) -> pd.DataFrame:

    df = df.copy()  # Avoid modifying the original DataFrame
    df['STD_gain'] = round(df['STD'].pct_change() * 100, 2)
    df['Middle_gain'] = round(df['Middle'].pct_change() * 100, 2)

    return df

TIME_RANGE_SORTED_1T = pd.date_range(start='09:15:00', end='15:30:00', freq='5s').strftime('%H:%M:%S').tolist()

def get_resume_index(df, while_time):
    idx = np.searchsorted(TIME_RANGE_SORTED_1T, while_time, side='right') - 1

    # Early exit if while_time is beyond last possible time
    if idx >= len(TIME_RANGE_SORTED_1T):
        return None

    rounded_arr = df['time'].values

    # Loop until a matching time is found in df
    while idx < len(TIME_RANGE_SORTED_1T):
        next_time = TIME_RANGE_SORTED_1T[idx]
        match_idx = np.flatnonzero(rounded_arr == next_time)

        if match_idx.size > 0:
            return match_idx[0] - len(df)

        idx += 1

    return None  # No future matching time found

results_ddff = []

# === Paths ===
sym_df_folder_path = r'D:\Offline Data\Processed Data\Get_symbol'
sym_df_folder_path = f.convert_csv_to_parquet(sym_df_folder_path)

folder_path = r'D:\Offline Data\Processed Data\Nifty_5_Second_ICICI\Nifty_5_Second_with_indicator_ICICI'
folder_path = f.convert_csv_to_parquet(folder_path)

su_1m_folder_path = r'D:\Offline Data\Processed Data\Nifty_1_Minute_ICICI\Nifty_1_Minute_with_indicator_ICICI'
su_1m_folder_path = f.convert_csv_to_parquet(su_1m_folder_path)

csv_files = [f for f in os.listdir(folder_path) if f.endswith('.parquet')]

# === Loop through files ===
for n in range(0, len(csv_files)):

    print(f"Processing File: {csv_files[n]}")

    current_date = csv_files[n].replace('.parquet', '').replace('_', '-')
    print(f"Current Date: {current_date}")

    # ===== 5 Second df Main =====
    file_path = os.path.join(folder_path, csv_files[n])
    _5s_df = pd.read_parquet(file_path)
    _5s_df = calculate_candle_gains(_5s_df)
    _5s_df['time'] = _5s_df['datetime'].str[-8:-2] + '00'
    _5s_symbol_groups = {k: v.reset_index(drop=True) for k, v in _5s_df.groupby('symbol')}    # Precompute Lookup

    # ===== 1 Minute supporting df =====
    su_1m_file_path = os.path.join(su_1m_folder_path, csv_files[n])
    _su_1m_df = pd.read_parquet(su_1m_file_path)
    su_1m_symbol_groups = {k: v.reset_index(drop=True) for k, v in _su_1m_df.groupby('symbol')}    # Precompute Lookup

    # ===== Find symbol =====
    sym_file_path = os.path.join(sym_df_folder_path, csv_files[n])
    sym_df = pd.read_parquet(sym_file_path)
    time_to_symbol = dict(zip(sym_df['time'], sym_df['symbol']))    # Precompute Lookup

    # -----------------------------

    while_time = "09:15:00"

    entry_i = None                  # Backtesting

    symbol_change_Flag = False   
    symbol_change_Flag_symbol = None

    while True:
        current_symbol = symbol_change_Flag_symbol if symbol_change_Flag else time_to_symbol.get(while_time)
        # print(current_symbol, symbol_change_Flag_symbol, symbol_change_Flag, time_to_symbol.get(while_time), while_time)
        symbol_change_Flag = False   

        df = _5s_symbol_groups.get(current_symbol)
        length_df = -len(df)
        # print(length_df)

        su_1m_df = su_1m_symbol_groups.get(current_symbol)
        su_1m_dict = dict(zip(su_1m_df['datetime'].values, su_1m_df.to_dict('records')))
        keys = list(su_1m_dict.keys())
        length_su_1m_df = len(su_1m_df)

        datetime_arr = df['datetime'].values
        symbol_arr = df['symbol'].values
        open_arr = df['open'].values
        high_arr = df['high'].values
        low_arr = df['low'].values
        close_arr = df['close'].values
        open_interest_arr = df['open_interest'].values
        volume_arr = df['volume'].values
        STD_arr = df['STD'].values
        Middle_arr = df['Middle'].values
        Upper_arr = df['Upper'].values
        Lower_arr = df['Lower'].values
        pctB_arr = df['%B'].values
        STD_gain_arr = df['STD_gain'].values
        Middle_gain_arr = df['Middle_gain'].values
        time_arr = df['time'].values

        resume_index = get_resume_index(df, while_time) if while_time != "09:15:00" else -len(df)

        entry_signal_is = False             # Backtesting
        # print("a", while_time, "--------------------------------------")

        for i in range(resume_index, 0):
            # print("i am hear 001")
            # print(datetime_arr[i])
            while_time = datetime_arr[i][-8:] 

            if symbol_change_Flag and not entry_signal_is:
                break

            next_symbol = time_to_symbol.get(shift_time(while_time, minutes=-1))

            if next_symbol != symbol_arr[i] and next_symbol is not None:
                if not entry_signal_is:
                    # print("Symbol Change", symbol_arr[i], next_symbol)
                    symbol_change_Flag_symbol = next_symbol
                    symbol_change_Flag = True
                    break
                else:
                    symbol_change_Flag_symbol = next_symbol
                    symbol_change_Flag = True

            target_time = datetime_arr[i][:-2] + '00'

            if target_time[-8:] > "09:17:00":
                idx_1m = keys.index(target_time) - length_su_1m_df

                # row = su_1m_dict[keys[idx_1m]] # it's have look ahead bias
                prev_row_1 = su_1m_dict[keys[idx_1m - 1]]
                prev_row_2 = su_1m_dict[keys[idx_1m - 2]]
                prev_row_3 = su_1m_dict[keys[idx_1m - 3]]
                # print(datetime_arr[i])

                # ==========================================
                # ==========================================
                # Backtesting model

                entry_signal_is = True

                # print("---- Entry ----", symbol_arr[i][-5:])
                # print(datetime_arr[i])

                entry_i = i

                # main_df model
                entry_datetime = datetime_arr[i]
                entry_symbol = symbol_arr[i]
                entry_open = open_arr[i]
                entry_high = high_arr[i]
                entry_low = low_arr[i]
                entry_close = close_arr[i]
                entry_open_interest = open_interest_arr[i]
                entry_volume = volume_arr[i]
                entry_STD = STD_arr[i]
                entry_Middle = Middle_arr[i]
                entry_Upper = Upper_arr[i]
                entry_Lower = Lower_arr[i]
                entry_pctB = pctB_arr[i]

                # =====================================================

                sl_1 = close_arr[entry_i] * 0.99
                sl_2 = close_arr[entry_i] * 0.985
                sl_3 = close_arr[entry_i] * 0.98

                tp_1 = close_arr[entry_i] * 1.10
                tp_2 = close_arr[entry_i] * 1.25
                tp_3 = close_arr[entry_i] * 1.50

                allex = 6       # how many exit
                allex_n = 0

                exit_sl1, exit_sl1_index, exit_sl1_time = None, 0, None
                exit_sl2, exit_sl2_index, exit_sl2_time = None, 0, None
                exit_sl3, exit_sl3_index, exit_sl3_time = None, 0, None

                exit_tp1, exit_tp1_index, exit_tp1_time = None, 0, None
                exit_tp2, exit_tp2_index, exit_tp2_time = None, 0, None
                exit_tp3, exit_tp3_index, exit_tp3_time = None, 0, None

                # print(i, "--------------------------------")

                next_symbol = time_to_symbol.get(shift_time(while_time, minutes=-1))

                if next_symbol != symbol_arr[i] and next_symbol is not None:
                    winsound.PlaySound("alarm.wav", winsound.SND_FILENAME | winsound.SND_ASYNC)
                    print("\033[1;31mSymbol error\033[0m")
                    raise Exception("\033[1;31mSymbol error\033[0m")

                for j in range(i+1, 0):

                    # ----- stoploss_1  
                    # =====================================================
                    if (low_arr[j] < sl_1 or j == -1) and exit_sl1 is None:
                        # print("---- stoploss_1 ----")
                        exit_sl1 = (close_arr[j] if j == -1 else sl_1) * 0.9975     # Charging 0.25% brokerage
                        exit_sl1_time = datetime_arr[j]
                        exit_sl1_index = j
                        allex_n += 1

                    # ----- stoploss_2  
                    # =====================================================
                    if (low_arr[j] < sl_2 or j == -1) and exit_sl2 is None:
                        # print("---- stoploss_2 ----")
                        exit_sl2 = (close_arr[j] if j == -1 else sl_2) * 0.9975     # Charging 0.25% brokerage
                        exit_sl2_time = datetime_arr[j]
                        exit_sl2_index = j
                        allex_n += 1

                    # ----- stoploss_3  
                    # =====================================================
                    if (low_arr[j] < sl_3 or j == -1) and exit_sl3 is None:
                        # print("---- stoploss_3 ----")
                        exit_sl3 = (close_arr[j] if j == -1 else sl_3) * 0.9975     # Charging 0.25% brokerage
                        exit_sl3_time = datetime_arr[j]
                        exit_sl3_index = j
                        allex_n += 1

                    # ----- target_1  
                    # =====================================================
                    if (high_arr[j] > tp_1 or j == -1) and exit_tp1 is None:
                        # print("---- target_1 ----")
                        exit_tp1 = (close_arr[j] if j == -1 else tp_1) * 0.9975     # Charging 0.25% brokerage
                        exit_tp1_time = datetime_arr[j]
                        exit_tp1_index = j
                        allex_n += 1

                    # ----- target_2  
                    # =====================================================
                    if (high_arr[j] > tp_2 or j == -1) and exit_tp2 is None:
                        # print("---- target_2 ----", symbol_arr[j][-5:])
                        exit_tp2 = (close_arr[j] if j == -1 else tp_2) * 0.9975     # Charging 0.25% brokerage
                        exit_tp2_time = datetime_arr[j]
                        exit_tp2_index = j
                        allex_n += 1

                    # ----- target_2  
                    # =====================================================
                    if (high_arr[j] > tp_3 or j == -1) and exit_tp3 is None:
                        # print("---- target_3 ----", symbol_arr[j][-5:])
                        exit_tp3 = (close_arr[j] if j == -1 else tp_3) * 0.9975     # Charging 0.25% brokerage
                        exit_tp3_time = datetime_arr[j]
                        exit_tp3_index = j
                        allex_n += 1

                    if allex == allex_n:
                        # print(allex, allex_n)
                        # print("i am hear 002")

                        entry_signal_is = False

                        results_ddff.append({
                            'entry_datetime': entry_datetime,
                            'entry_symbol': entry_symbol,
                            'combi_entry_open': entry_open,
                            'combi_entry_high': entry_high,
                            'combi_entry_low': entry_low,
                            'combi_entry_close': entry_close,
                            'combi_entry_open_interest': entry_open_interest,
                            'combi_entry_volume': entry_volume,
                            'combi_entry_STD': entry_STD,
                            'combi_entry_Middle': entry_Middle,
                            'combi_entry_Upper': entry_Upper,
                            'combi_entry_Lower': entry_Lower,
                            'combi_entry_pctB': entry_pctB,
                            'exit_sl1': exit_sl1, 'exit_sl1_pnl': round((exit_sl1 - entry_close) / entry_close * 100, 2), 'exit_sl1_index': abs(entry_i) - abs(exit_sl1_index), 'exit_sl1_time': exit_sl1_time,
                            'exit_sl2': exit_sl2, 'exit_sl2_pnl': round((exit_sl2 - entry_close) / entry_close * 100, 2), 'exit_sl2_index': abs(entry_i) - abs(exit_sl2_index), 'exit_sl2_time': exit_sl2_time,
                            'exit_sl3': exit_sl3, 'exit_sl3_pnl': round((exit_sl3 - entry_close) / entry_close * 100, 2), 'exit_sl3_index': abs(entry_i) - abs(exit_sl3_index), 'exit_sl3_time': exit_sl3_time,
                            'exit_tp1': exit_tp1, 'exit_tp1_pnl': round((exit_tp1 - entry_close) / entry_close * 100, 2), 'exit_tp1_index': abs(entry_i) - abs(exit_tp1_index), 'exit_tp1_time': exit_tp1_time,
                            'exit_tp2': exit_tp2, 'exit_tp2_pnl': round((exit_tp2 - entry_close) / entry_close * 100, 2), 'exit_tp2_index': abs(entry_i) - abs(exit_tp2_index), 'exit_tp2_time': exit_tp2_time,
                            'exit_tp3': exit_tp3, 'exit_tp3_pnl': round((exit_tp3 - entry_close) / entry_close * 100, 2), 'exit_tp3_index': abs(entry_i) - abs(exit_tp3_index), 'exit_tp3_time': exit_tp3_time,
                            })

                        # ---------- Plotting -----------

                        # # main 5 second
                        # starting, ending = f.starting_ending(entry_i, exit_index, length_df)
                        # df_for_plot = df[starting:ending].copy()

                        # f.plot_chart(df_for_plot,
                        #             entry_time = entry_time, exit_time = exit_time,
                        #             entry_price = entry_price, exit_price = exit_price,
                        #             position = "Long",
                        #             pnl = pnl,
                        #             trade_n = 0,
                        #             supertrend_start_time = entry_time,
                        #             plot_bollinger = True,
                        #             plot_pctb = True,
                        #             plot_supertrend = False,
                        #             plot_entry_exit = True,
                        #             save_chart = False)

                        break # break loop
                        
                # ==========================================
                # ==========================================

        if i == -1:
            if len(results_ddff) < 1:
                raise ValueError("\033[1;30mManual error :\033[0m 'allex' variable is incorrect")
            break

main_df = pd.DataFrame(results_ddff)

exit_keywords = [col for col, is_true in main_df.isnull().all().items() if is_true]
main_df = main_df.drop([col for col in main_df.columns if any(kw in col for kw in exit_keywords)],axis=1)

main_df['entry_date'] = main_df['entry_datetime'].str[:-9]
main_df['entry_time'] = main_df['entry_datetime'].str[-8:]
main_df['is_last_candle'] = main_df['entry_date'] == main_df['entry_date'].shift(-1)

os.makedirs('Data', exist_ok=True)
main_df.to_csv('Data/backtest_model_results.csv', index=False)


All files already converted to parquet format
All files already converted to parquet format
All files already converted to parquet format
Processing File: 2025_01_01.parquet
Current Date: 2025-01-01
Processing File: 2025_01_02.parquet
Current Date: 2025-01-02
Processing File: 2025_01_03.parquet
Current Date: 2025-01-03
Processing File: 2025_01_06.parquet
Current Date: 2025-01-06
Processing File: 2025_01_07.parquet
Current Date: 2025-01-07
Processing File: 2025_01_08.parquet
Current Date: 2025-01-08
Processing File: 2025_01_09.parquet
Current Date: 2025-01-09
Processing File: 2025_01_10.parquet
Current Date: 2025-01-10
Processing File: 2025_01_13.parquet
Current Date: 2025-01-13
Processing File: 2025_01_14.parquet
Current Date: 2025-01-14
Processing File: 2025_01_15.parquet
Current Date: 2025-01-15
Processing File: 2025_01_16.parquet
Current Date: 2025-01-16
Processing File: 2025_01_17.parquet
Current Date: 2025-01-17
Processing File: 2025_01_20.parquet
Current Date: 2025-01-20
Processi